# Final Project: Partial Least-Squares Regression in Python


In [ ]:
#shiyishi

In [59]:
from PIL import Image
import numpy as np
import numpy.linalg 
import re
import sys, os
import functools
import glob
numpy.set_printoptions(precision=3)

## Abstract 
 
The partial least-squares regression method(PLS) is a statistical approach to make a linear regression model by projecting the predicted variables and the observable variables to a new space. By combining features of principal component analysis and multiple regression, PLS is more robust than principal component regression and simple OLS regression when we need a large set of variables with a few samples. 
In this final project, we implemented, tested and optimized the algorithm in the paper “PARTIAL LEAST SQUARES REGRESSION: A TUTORIAL”[1]. Then we offered a comparison of the code with other popular codes of PLS, followed by sevaral applications of the code in fields of chemistry and economy. 

## 1. Introduction  

### 1.1 Paper Background

•	State the research paper “PARTIAL LEAST SQUARES REGRESSION: A TUTORIAL”. 

•	Literature review

#### Literature review
the review paper "Overview and Recent Advances in Partial Least
    Squares" by Roman Rosipal and Nicole Kramer, LNCS, 2006, for
    additional information related to the different variants of PLS and
    for citations to more recent work related to the algorithm.

### 1.2 Algorithm Description

*	Describe the concept of the algorithm and why it is interesting and/or useful. 
*	Describe the mathematical basis of the algorithm.  

### 1.3 Applications

*	Describe the importance of the methods in the paper

PLS is useful when you are dealing with data that have following characteristic:
 * make multidimensional predictions from multidimensional observations.
 * the dimensionality of the observation space is large.
 * the data you have available for constructing a prediction model is rather limited.  

Compared with PLS, The more traditional multiple linear regression     (MLR) algorithms are likely to become numerically unstable under     these conditions("multicollinearity problem" caused by strong correlations between the different predictor variables).

 

## 2.	Implementation 

Implement the algorithm as a Python function or family of functions.
*	Does it run?
*	Is it correct? How do you know? Use of tests
*	Is it written cleanly and efficiently?

### 2.1 Multiple Linear Regression(MLR)

#### 2.1.1 MLR with only one dependent variable 

Consider MLR with only one dependent variable

#### 2.1.2 MLR with more than one dependent variable 

Consider MLR with more than one dependent variable

#### 2.1.3 When encounter the multicollinearity problem 

* Reducing the dimensionality N of the space of the predictor variables through the application of Principal Components Analysis(PCA) to your data.

* using the method of Partial Least Squares (PLS).

### 2.2 Principal Component Analysis(PCA)

PCA focuses solely on the space defined by the predictor variables and gives you a small orthogonal set of directions in that space that explain the most significant variations in just the space defined by x1, x2, ...., xN.Subsequently, you can use the principal directions (these will be linear combinations of the original predictor variables) for making predictions.  

#### 2.2.1  PCA code

In [1]:
## create two 3x20 datasets
import numpy as np

np.random.seed(234234) # random seed for consistency
 
mu_vec1 = np.array([0,0,0])
cov_mat1 = np.array([[1,0,0],[0,1,0],[0,0,1]])
class1_sample = np.random.multivariate_normal(mu_vec1, cov_mat1, 20).T
assert class1_sample.shape == (3,20), "The matrix has not the dimensions 3x20"
 
mu_vec2 = np.array([1,1,1])
cov_mat2 = np.array([[1,0,0],[0,1,0],[0,0,1]])
class2_sample = np.random.multivariate_normal(mu_vec2, cov_mat2, 20).T
assert class1_sample.shape == (3,20), "The matrix has not the dimensions 3x20"

In [2]:
## Using the whole dataset without class labels
all_samples = np.concatenate((class1_sample, class2_sample), axis=1)
assert all_samples.shape == (3,40), "The matrix has not the dimensions 3x40"

In [3]:
## d-dimensional mean vector
mean_x = np.mean(all_samples[0,:])
mean_y = np.mean(all_samples[1,:])
mean_z = np.mean(all_samples[2,:])
 
mean_vector = np.array([[mean_x],[mean_y],[mean_z]])
 
print('Mean Vector:\n', mean_vector)



Mean Vector:
 [[ 0.57786468]
 [ 0.34875596]
 [ 0.52071362]]


In [4]:
## Computing the Scatter Matrix
scatter_matrix = np.zeros((3,3))
for i in range(all_samples.shape[1]):
    scatter_matrix += (all_samples[:,i].reshape(3,1)\
         - mean_vector).dot((all_samples[:,i].reshape(3,1) - mean_vector).T)
print('Scatter Matrix:\n', scatter_matrix)  

Scatter Matrix:
 [[ 52.61055245  15.06845363   4.07624697]
 [ 15.06845363  45.56755824   0.92729735]
 [  4.07624697   0.92729735  67.84643492]]


In [5]:
## Computing the Covariance Matrix which is an alternative expression of to the scatter matrix
cov_mat = np.cov([all_samples[0,:],all_samples[1,:],all_samples[2,:]])
print('Covariance Matrix:\n', cov_mat)

Covariance Matrix:
 [[ 1.34898852  0.38637061  0.10451915]
 [ 0.38637061  1.16839893  0.02377686]
 [ 0.10451915  0.02377686  1.73965218]]


In [6]:
# eigenvectors and eigenvalues for the from the scatter matrix
eig_val_sc, eig_vec_sc = np.linalg.eig(scatter_matrix)
 
# eigenvectors and eigenvalues for the from the covariance matrix
eig_val_cov, eig_vec_cov = np.linalg.eig(cov_mat)
 
for i in range(len(eig_val_sc)):
    eigvec_sc = eig_vec_sc[:,i].reshape(1,3).T
    eigvec_cov = eig_vec_cov[:,i].reshape(1,3).T
    assert eigvec_sc.all() == eigvec_cov.all(), 'Eigenvectors are not identical'
 
    print('Eigenvector {}: \n{}'.format(i+1, eigvec_sc))
    print('Eigenvalue {} from scatter matrix: {}'.format(i+1, eig_val_sc[i]))
    print('Eigenvalue {} from covariance matrix: {}'.format(i+1, eig_val_cov[i]))
    print('Scaling factor: ', eig_val_sc[i]/eig_val_cov[i])
    print(40 * '-')

Eigenvector 1: 
[[ 0.62563478]
 [-0.77829547]
 [-0.05326609]]
Eigenvalue 1 from scatter matrix: 33.51820750301696
Eigenvalue 1 from covariance matrix: 0.8594412180260762
Scaling factor:  39.0
----------------------------------------
Eigenvector 2: 
[[-0.63625553]
 [-0.54857976]
 [ 0.54243815]]
Eigenvalue 2 from scatter matrix: 62.12738429750416
Eigenvalue 2 from covariance matrix: 1.5930098537821578
Scaling factor:  39.0
----------------------------------------
Eigenvector 3: 
[[ 0.45139786]
 [ 0.30547733]
 [ 0.83840538]]
Eigenvalue 3 from scatter matrix: 70.37895381131396
Eigenvalue 3 from covariance matrix: 1.8045885592644608
Scaling factor:  39.0
----------------------------------------


In [7]:
## Checking the eigenvector-eigenvalue calculation
for i in range(len(eig_val_sc)):
    eigv = eig_vec_sc[:,i].reshape(1,3).T
    np.testing.assert_array_almost_equal(scatter_matrix.dot(eigv), eig_val_sc[i] * eigv, 
                                         decimal=6, err_msg='', verbose=True)

#### 2.2.2 PCA example

In [8]:
## Principal Component Regression Example
## Generating model with 2 factors and 4 explanatory variables
## Try out partial correlation for dropping (or adding) factors (variable importance)
###############
## Apply algorithm for partial least squares as an alternative to PCR 
###############

import numpy as np
from numpy.testing import assert_array_almost_equal
import statsmodels.api as sm
from statsmodels.sandbox.tools import pca
from statsmodels.sandbox.tools.cross_val import LeaveOneOut


# Example: principal component regression
nobs = 1000
f0 = np.c_[np.random.normal(size=(nobs,2)), np.ones((nobs,1))]
f2xcoef = np.c_[np.repeat(np.eye(2),2,0),np.arange(4)[::-1]].T
f2xcoef = np.array([[ 1.,  1.,  0.,  0.],
                    [ 0.,  0.,  1.,  1.],
                    [ 3.,  2.,  1.,  0.]])
f2xcoef = np.array([[ 0.1,  3.,  1.,    0.],
                    [ 0.,  0.,  1.5,   0.1],
                    [ 3.,  2.,  1.,    0.]])
x0 = np.dot(f0, f2xcoef)
x0 += 0.1*np.random.normal(size=x0.shape)
ytrue = np.dot(f0,[1., 1., 1.])
y0 = ytrue + 0.1*np.random.normal(size=ytrue.shape)

xred, fact, eva, eve  = pca(x0, keepdim=0)
print(eve)
print(fact[:5])
print(f0[:5])

[[-0.02996493  0.01657949  0.47144762 -0.88122889]
 [-0.91625175  0.39855097  0.00320356  0.04036806]
 [-0.39942215 -0.91429223 -0.05778477 -0.03453397]
 [-0.0068391  -0.07037022  0.87999307  0.46969507]]
[[-2.56867619 -1.84379912 -0.08842251 -0.14316232]
 [ 1.06347852  1.73614937  0.03640215 -0.00571607]
 [ 1.34018561  0.21983743 -0.2754377  -0.06737226]
 [-2.75758621 -0.78687507  0.09278904  0.07681938]
 [-0.06785456  1.00988657 -0.01006479 -0.00437881]]
[[ 0.57041486  1.33336693  1.        ]
 [-0.01685332 -1.26126422  1.        ]
 [-0.35731031 -0.24072366  1.        ]
 [ 0.77175347  0.79462462  1.        ]
 [ 0.15355514 -0.76336247  1.        ]]


In [10]:
import statsmodels.api as sm

res = sm.OLS(y0, sm.add_constant(x0, prepend=False)).fit()
print('OLS on original data')
print(res.params)
print(res.aic)
print(res.rsquared)

#print 'OLS on Factors'
#for k in range(x0.shape[1]):
#    xred, fact, eva, eve  = pca(x0, keepdim=k, normalize=1)
#    fact_wconst = sm.add_constant(fact)
#    res = sm.OLS(y0, fact_wconst).fit()
#    print 'k =', k
#    print res.params
#    print 'aic:  ', res.aic
#    print 'bic:  ', res.bic
#    print 'llf:  ', res.llf
#    print 'R2    ', res.rsquared
#    print 'R2 adj', res.rsquared_adj

print('OLS on Factors')
results = []
xred, fact, eva, eve  = pca(x0, keepdim=0, normalize=1)
for k in range(0, x0.shape[1]+1):
    #xred, fact, eva, eve  = pca(x0, keepdim=k, normalize=1)
    # this is faster and same result
    fact_wconst = sm.add_constant(fact[:,:k], prepend=False)
    res = sm.OLS(y0, fact_wconst).fit()
##    print 'k =', k
##    print res.params
##    print 'aic:  ', res.aic
##    print 'bic:  ', res.bic
##    print 'llf:  ', res.llf
##    print 'R2    ', res.rsquared
##    print 'R2 adj', res.rsquared_adj
    prederr2 = 0.
    for inidx, outidx in LeaveOneOut(len(y0)):
        resl1o = sm.OLS(y0[inidx], fact_wconst[inidx,:]).fit()
        #print data.endog[outidx], res.model.predict(data.exog[outidx,:]),
        prederr2 += (y0[outidx] - resl1o.predict(fact_wconst[outidx,:]))**2.
    results.append([k, res.aic, res.bic, res.rsquared_adj, prederr2])

results = np.array(results)
print(results)
print('best result for k, by AIC, BIC, R2_adj, L1O')
print(np.r_[(np.argmin(results[:,1:3],0), np.argmax(results[:,3],0),
             np.argmin(results[:,-1],0))])

OLS on original data
[-0.07001365  0.1139064   0.66138757  0.01597094  0.31570395]
-1338.95919821
0.992216422701
OLS on Factors
[[  0.00000000e+00   3.50878004e+03   3.51368780e+03   1.11022302e-16
    1.95600470e+03]
 [  1.00000000e+00   2.44054326e+03   2.45035877e+03   6.56729076e-01
    6.72063282e+02]
 [  2.00000000e+00  -1.33897450e+03  -1.32425124e+03   9.92169669e-01
    1.53480466e+01]
 [  3.00000000e+00  -1.33924906e+03  -1.31961804e+03   9.92179616e-01
    1.53433643e+01]
 [  4.00000000e+00  -1.33895920e+03  -1.31442042e+03   9.92185132e-01
    1.53531464e+01]]
best result for k, by AIC, BIC, R2_adj, L1O
[3 2 4 3]


In [ ]:
from statsmodels.iolib.table import (SimpleTable, default_txt_fmt,
                        default_latex_fmt, default_html_fmt)

headers = 'k, AIC, BIC, R2_adj, L1O'.split(', ')
numformat = ['%6d'] + ['%10.3f']*4 #'%10.4f'
txt_fmt1 = dict(data_fmts = numformat)
tabl = SimpleTable(results, headers, None, txt_fmt=txt_fmt1)

print("PCA regression on simulated data,")
print("DGP: 2 factors and 4 explanatory variables")
print(tabl)
print("Notes: k is number of components of PCA,")
print("       constant is added additionally")
print("       k=0 means regression on constant only")
print("       L1O: sum of squared prediction errors for leave-one-out")

PCA regression on simulated data,
DGP: 2 factors and 4 explanatory variables
  k       AIC        BIC       R2_adj      L1O    
--------------------------------------------------
     0   3508.780   3513.688      0.000   1956.005
     1   2440.543   2450.359      0.657    672.063
     2  -1338.975  -1324.251      0.992     15.348
     3  -1339.249  -1319.618      0.992     15.343
     4  -1338.959  -1314.420      0.992     15.353
--------------------------------------------------
Notes: k is number of components of PCA,
       constant is added additionally
       k=0 means regression on constant only
       L1O: sum of squared prediction errors for leave-one-out


### 2.3 Principal Component Regression(PCR)

### 2.4 Partial Least-Squares Regression

PLS also gives you a small set of principal directions (also called principal components) in the space defined  by the original predictor variables.  
However, the directions yielded by PLS also take into account the observed variations in the space defined by the predicted variables y1, y2, ...., yM.  
So if a principal component cannot explain the variations in the space defined by the predicted variables y1, y2, ..., yM, it will be ignored by PLS.

#### 2.4.1 Model building

###### Notations:

* X:
The matrix formed by the recorded values for the predictor variables x1, x2, ..., xN is typically denoted X.
Each row of X is one observation record and each column of X stands
for one predictor variable.  

* Y:
The values for the predicted variables y1, y2, ..., yM are also placed in a matrix that is typically denoted Y.  
Each column of Y stands for one predicted variable and each row of Y for the values for all the predicted variables using the corresponding row of X for prediction.

* So if we have I observation records available to us, X is a matrix of size IxN and Y is a matrix of size IxM.

* Each principal component of X, usually referred to as a latent vector of X, is represented by t. 
The matrix whose columns are the latent vectors t is typically denoted T. 
If the p is the number of latent vectors discovered for X, then T is of size Ixp.  

* Along the same lines, each latent vector of Y is typically denoted u and the matrix formed by these latent vectors is denoted U. 
In most variants of the PLS algorithm, the latent vectors for X and Y are discovered conjointly.
In such cases, U will also be of size Ixp.


* Loadings: Each latent vector t is a weighted linear combination of the columns of X and each latent vector u is a weighted linear combination of the columns of Y.  The weights that go into these
    combinations can also be thought of as vectors. These weights are
    referred to as loadings.


* The loading vectors for X are represented
    by p and those for Y by q.  The matrix formed by the p vectors
    represented by P and by the q vectors by Q.


* Two more matrices important to the discovery of the latent vectors
    for X and Y are W and C.  As mentioned earlier, a latent vector t
    is merely a weighted linear combination of the columns of X. For a
    calculated t, we represent these weights in the form of a vector w,
    and all the w vectors (for the different t vectors) taken together
    constitute the matrix W.  



* By the same token, a latent vector u is
    is a weighted linear combination of the columns of Y and, for a
    given u, we represent these weights in the form of a vector c.  All
    the different vectors c (for the different u vectors) constitute
    the matrix C.


* A matrix of B: regression coefficients.  Once we have B, given a test data matrix Xtest, we can     make the prediction Ytest = (Xtest * B).  

#### 2.4.2 The mathematical foundation of the algorithms  

#### Main steps for PLS  algorithm.  

Goal: To find the latent vector t in the column space of X and the latent vector u in the column space of Y so that the covariance     (t' * u)/I is maximized.  

* t' is the transpose of t;
* I is the number of rows in X (and in Y)

* Starting with a random guess for u, we iterate through the following eights steps until achieving the termination condition stated in the last step:

##### Step (1) w = X' * u

where X' is the transpose of X and u is the current value of the latent vector for Y.  

We will use the elements of the vector w thus obtained as weights for creating a linear combination of the columns of X as a new candidate for t.

##### Step(2)  w =  w / ||w||

where ||w|| is the norm of the vector w.  
That is, ||w|| = sqrt(w' * w).  
This step normalizes the magnitude of w to 1.

##### Step (3)  t = X * w

Now we have a new approximation to t.  For PLS, we also     normalize t as we normalized w in the previous step.


##### Step (4)  c = Y' * t

We awill use the elements of the vector c thus obtained as         weights for creating a linear combination of the columns           of Y for a new candidate as Y's latent vector u.

##### Step (5)  c = c / ||c||
Normalize c just as we normalized w.

##### Step (6)  u_old  =  u
We store away the currently used value for u

##### Step (7)  u = Y * c
Now we have a new candidate for the latent vector for Y.


##### Step (8)  terminate the iterations if 

          ||u - u_old||<tol, where tol is a user-specified value.


##### Calculate p and q according to the obtained t and u

Once we have a vector t from the column space of X and a vector u  from the column space of Y, we need to figure out what weighted  linear combination of the columns of X would result t and what  weighted linear combination of the column vectors of Y would result  in Y.  Referring to these weights by the vectors p and q, we calculate

         p  =  (X' * t) / ||t||

         q  =  (Y' * u) / ||u||


* If this is our first pair (t,u) of latent vectors, we initialize the matrices T and U by setting them to the column vectors t and u,respectively.  

* If this is not the first pair (t,u), we augment T and U by the additional column vectors t and u, respectively.  The same goes for the vectors p and q vis-a-vis the matrices P and Q.


##### Deflation:
After the calculation of each pair (t,u) of latent vectors, it's
time to subtract out from X the contribution made to it by t, and
to subtract out from Y the contribution made to it by u. 
This step,called deflation as mentioned previously, is implemented as follows:

For PLS( ):

         X  =  X - (t * p')

         Y  =  Y - b * t * c'

         where b = t' * u. 


For PLS1(  ) and PLS2(  ):


         X  =  X - (t * p')

         Y  =  Y - (t * t' * Y) / ||t||


This process of first finding the covariance maximizing latent  vectors t and u from the column spaces of X and Y, respectively,    and subsequently deflating X and Y as shown above is continued     until there is not much left of the matrices X and Y. 

 

###### Calculate matrix B

After the latent vectors of X and Y have been extracted, the last   step is the calculation of the matrix B of regression coefficients. For the PLS() implementation, this calculation involves

                 _                  
        B =  (P')  * diag[b1, b2, ...., bp] * C'

          
where inv((P')) is the pseudo-inverse of the transpose of P. The  second term above is the diagonal matrix formed by the b values     computed after the calculation of each pair (t,u) as mentioned     earlier. C' is the transpose of the C matrix.


For PLS1() and PLS2(), the B matrix is calculated through the following formula:

        B = W * (P' * W)^-1 * C'

where (P' * W)^-1 is the matrix inverse of the product of the two  matrices P' and W.   

##### Predictions

After you have calculated the B matrix, you are ready to make predictions in the future as new row vectors for the X matrix come along. Let's use the notation Xtest to denote the new data     consisting of the observed values for the predictor variables.
Your prediction would now consist of

                      Ytest =  Xtest * B

#### 2.4.3 The code: PLS（ ）

In [90]:
# centralize the data

def centralize(X,Y):
    X=X-np.mean(X)
    Y=Y-np.mean(Y)
    return (X,Y)

def PLS(X,Y,tol=1e-6):

    N = X.shape[0]
    u = numpy.random.rand(1,N)
    u = numpy.asmatrix(u).T
    
    #initialize T，U,W,C
    T=U=W=C=None 

    #initialize P,Q,Bdiag
    P=Q=Bdiag=None

    i=0
    while(True):                             
        j = 0
        while (True):
            #Step(1)
            w = X.T * u   
            #Step(2) Normalizes the magnitude of w to 1.
            w = w / numpy.linalg.norm(w)
            #Step(3) a new approximation to t 
            t = X * w  
            t = t / numpy.linalg.norm(t) #Normalize t
            #Step(4) Obtain c
            c = Y.T * t
            #Step(5) Normalize c 
            c = c / numpy.linalg.norm(c)        
            #Step(6) Store the currently used value for u
            u_old = u
            #Step(7) A new candidate for the latent vector for Y. 
            u = Y * c
            #Step(8) Terminate the iterations if error is small enough
            error = numpy.linalg.norm(u - u_old)
            if error < tol:
                break

            j += 1 

        #Obtain matrix b
        b = t.T * u
        b = b[0,0]
        
        if T is None:
            T = t
        else:
            T = numpy.hstack((T,t))
        if U is None:
            U = u
        else:
            U = numpy.hstack((U,u))
        if W is None:
            W = w
        else:
            W = numpy.hstack((W,w))
        if C is None:
            C = c
        else:
            C = numpy.hstack((C,c))
     
        p = X.T * t / (np.linalg.norm(t) ** 2)
        q = Y.T * u / (np.linalg.norm(u) ** 2)

        if P is None:
            P = p
        else:
            P = numpy.hstack((P,p))
        if Q is None:
            Q = q
        else:
            Q = numpy.hstack((Q,q))
        if Bdiag is None:
            Bdiag = [b]
        else:
            Bdiag.append(b)
         
        X_old = X
        Y_old = Y
        X = X - t * p.T
        Y = Y - b * t * c.T
        i += 1

        if numpy.linalg.norm(X) < 0.001: break

        #Obtain B
        B = numpy.diag(Bdiag)
        B = numpy.asmatrix(B) 
        B = numpy.linalg.pinv(P.T) * B * C.T
        
        #Prediction based on the original X:
        
        Y_predicted = centralize(X,Y)[0] * B
        print("\nY_predicted from the original X:")
        print(Y_predicted)

        Y_predicted_with_mean = Y_predicted + np.mean(Y)
        print("\nThe predicted Y with the original Y's column-wise mean added:")
        print(Y_predicted_with_mean)
         
        return B


In [91]:
# Data
X = np.array([[0., 0., 1.],[1.,0.,0.], [2.,2.,2.], [2.,5.,4.]])
Y = np.array([[0.1, -0.2], [0.9, 1.1], [6.2, 5.9], [11.9, 12.3]])

#Testing
PLS(X,Y)


Y_predicted from the original X:
[[-0.124 -0.126]
 [-0.147 -0.15 ]
 [-0.143 -0.145]
 [-0.136 -0.139]]

The predicted Y with the original Y's column-wise mean added:
[[-0.297 -0.299]
 [-0.32  -0.322]
 [-0.315 -0.318]
 [-0.309 -0.311]]


matrix([[ 0.646,  0.656],
        [ 1.268,  1.288],
        [ 1.076,  1.093]])

## 3.	Optimization


##### The different versions of the algorithm differ in the following ways: 

* (1) As each pair of latent
    vectors, t from X and u from Y, is discovered, the different
    versions of PLS differ with regard to the deflation step. Recall
    that by deflation we mean how we subtract the influence of t from X
    and of u of Y before starting search for the next pair (t,u) of
    latent vectors. 

* (2) The different versions different with regard to
    the normalization of the latent vectors. 

*  (3) the different
    versions differ with regard to how the matrix B of the regression
    coefficients is calculated.  

 ##### Three variants of the PLS algorithm: PLS( ),PLS1( ),PLS2( )

 

*  PLS( ) Based on the description of the algorithm by Herve Abdi in
    the article "Partial Least Squares Regression and Projection on
    Latent Structure Regression," Computational Statistics, 2010.  (This particular     version generates the best regression results.)  


*  PLS1( ) and PLS2( ) are based on the description of the
    algorithm in the previously cited paper by Roman Rosipal and Nicole Kramer.  The PLS1() algorithm has a special role amongst all of the     variants of the partial least squares method --- it is meant     specifically for the case when the matrix Y consists of only one     column vector.  That is, we use PLS1( ) when there is just one  predictor variable.  

In [92]:
#PLS1

def PLS1(X,Y,tol=1e-6):
    if Y.shape[1] != 1:
        raise ValueError("PLS1 can only be called when the Y has only one column")
    X,Y=centralize(X,Y)
    T=U=W=C=P=Q=B=t=w=u=c=p=q=None        
    u = Y
    i = 0
    while (True):
        w = np.dot(X.T , u)
        w = w / numpy.linalg.norm(w)
        t = np.dot(X , w)
        c = np.dot(Y.T , t)
        c = c / numpy.linalg.norm(c)        
        u = np.dot(Y , c)
        if T is None:
            T = t
        else:
            T = numpy.hstack((T,t))
        if U is None:
            U = u
        else:
            U = numpy.hstack((U,u))
        if W is None:
            W = w
        else:
            W = numpy.hstack((W,w))
        p = np.dot(X.T,t/(numpy.linalg.norm(t)**2))
        q = np.dot(Y.T,u/(numpy.linalg.norm(u)**2))

        if P is None:
            P = p
        else:
            P = numpy.hstack((P,p))
        if Q is None:
            Q = q
        else:
            Q = numpy.hstack((Q,q))

        X_old = X
        Y_old = Y
        xdot=np.dot(t, p.T)
        X = X - xdot
        Y = Y - np.dot(xdot,Y)/(np.linalg.norm(t)**2)
        i += 1
        if np.linalg.norm(X) < 0.001: break
         
        B = W * ((P.T * W).I) * T.T * Y
       
        return B


## Summary

* Using C, C++ or Fortran give essentially identcial performance
* Of the JIT solutions:
    * Cython is the fastest but needs the extra work of type annotations
    * numba is almost as fast and simplest to use - just say jit(functiion)
    * numexpr is slightly slower and works best for small numpy expressions but is also very convenient
* A pure numpy solution also perfroms reasonably and will be the shortest solutoin (a one-liner in this case)
* The pure python approach is very slow, but serves as a useful template for converting to native langauge directly or via a JIT compiler
* Note that the fsatest alternatives are approximately 1000 times faster than the pure python version for the test problem with n=1000 and p=3.

### Recommendations for optimizing Python code

* Does a reliable fast implementiaont already exist? If so, consider using that
* Start with a numpy/python prototype - if this is fast enough, stop
* See if better use of vectoriazaiton via numpy will help
* Moving to native code:
    * Most Python devleopers will use Cython as the tool of choice. Cython can also be used to access/wrap C/C++ code
    * JIT compilation with numba is improving fast and may become competitive with Cython in the near future
    * If the function is "minimal", it is usually worth considering numexpr because there is almost no work to be done
    * Use C/C++/Fortran if you are fluent in those languages - you have seen how to call these functions from Python
* If appropriate, consider parallelization (covered in later session)
* As you optimize your code, remmeber:
    * Check that is is giving correct results!
    * Profile often - it is very hard to preidct the effect of an optimizaiton in general
    * Remember that your time is precious - stop when fast enough
    * If getting a bigger, faster machine will sovle the problem, that is sometimes the best solution

## 5. Application  

## 6.Comparison

## 7.	Reproducible analysis


## 8.	Reference 
